In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
import collections

In [24]:
def run_kmeans2(dataset):
    # features is the data matrix except for the last two columns
    X=dataset.iloc[:,0:-2]
              
    clean_Y = dataset.iloc[:,-2]
    tilde_Y = dataset.iloc[:,-1]

    values, counts = np.unique(clean_Y, return_counts=True)

    kmeans = KMeans(n_clusters=len(values))
    kmeans.fit(X, tilde_Y)
    identified_clusters = kmeans.fit_predict(X)

    # note that to better match the cluster Id to tilde_Y,
    # we could use hat_clean Y which obtained by current noise-robust method, but for the simple dataset , it may not necessary

    idx2 = _hungarian_match(identified_clusters, tilde_Y)
    #idx2 = _hungarian_match(identified_clusters, clean_Y)
    prime_Y = get_prime_Y(tilde_Y, identified_clusters, idx2[1])
    # yz: directly return prime_Y without using count_m
    return prime_Y

def _hungarian_match(flat_preds, flat_targets):
    # Based on implementation from IIC
    num_samples = flat_targets.shape[0]
    v, counts = np.unique(flat_preds, return_counts=True)

    num_k = len(v)
    num_correct = np.zeros((num_k, num_k))

    for c1 in range(num_k):
        for c2 in range(num_k):
            # elementwise, so each sample contributes once
            votes = int(((flat_preds == c1) * (flat_targets == c2)).sum())
            num_correct[c1, c2] = votes

    row, col = linear_sum_assignment(num_samples - num_correct)
    return row, col


def get_prime_Y(noisy_classes, pred_classes, mappings):
    prime_Y = np.zeros(len(noisy_classes))
    for i in range(len(pred_classes)):
        prime_Y[i] = mappings[pred_classes[i]]

    return prime_Y

2


,0,1,2
0,7679,1,1
1,152,1,1
2,17232,2,2
3,28829,2,2
4,9188,1,1


In [22]:


primeY =[]

primeY= run_kmeans2(data)

#res2 = (primeY == train_noisy_labels[:, 0])
res2 = (primeY == data.iloc[:,-1])
#print(res2)
count2 = collections.Counter(res2)
error_rate = count2[False]/(count2[False]+count2[True])
print(error_rate)

0.7165354330708661


In [2]:

data = pd.read_csv('datasets/iris/iris_train.csv')
from cdt.causality.graph import PC
pc_algorithm = PC()
result_graph = pc_algorithm.predict(data)
import matplotlib.pyplot as plt

pos = nx.nx_agraph.graphviz_layout(result_graph, prog='dot')
nx.draw(result_graph, pos, with_labels=True, node_size=1500, node_color='orange', font_size=10, font_weight='bold')
plt.show()


ImportError: R Package (k)pcalg/RCIT is not available. RCIT has to be installed from https://github.com/Diviyan-Kalainathan/RCIT